# Data Load


In [ ]:
import numpy as np
import pandas as pd
import os
import math
import random 

from scipy import stats #Analysis 
from scipy.stats import norm 

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [ ]:
# 한글 폰트가 깨지면 실행하세요! 
import matplotlib.font_manager as fm
fontlist = fm.findSystemFonts(fontpaths = None, fontext='ttf')
# 아래 주석을 지워서 폰트 리스트를 확인하고 한글 폰트를 font_path에 추가합니다
fontlist[:]

from matplotlib import font_manager, rc
#font_path = 'C:\\Users\\mtang\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
font_path = 'C:\\WINDOWS\\Fonts\\NanumGothicLight.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [ ]:
path = "./Dataset/"

data = pd.read_csv(path+"kwproja_data_location.csv")

# original data -> data
data

# EDA 

2,927,739 rows X 9 columns

- 매장 속성 정보
  - shop_code -> 식별자 feature, input feature로는 사용하지 않지만 분류를 위해서는 사용할 수 있을 것 같음  
  - shop_name -> input feature로는 사용하지 않음 (NLP deep learning 가능성 있음)
  - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> 군집화, labeling 필요
  - shop_type_big -> 15 category  -> 업종 (매출 영향성 있음)
  - shop_type_small -> 61 category

- 매출 정보
  - date -> 24 category, 201606~ 201805 까지의 data
  - monthly_gain / avearge_sale_price = 한달 총 판매수

- 매출 통계 정보-> X


##### monthly_gain과 average_sale_price 중 어느 것을 y값으로 둘 것인가? 
- 월매출 예측 문제로 가정하고 montly_gain 을 y값으로 예측하는 모델 만들기

##### shop_code는 input feature에 넣어야 하는가?
- 특별한 브랜드가 y값을 결정하는 과적합 요소가 될 수 있으므로
- X 에서 shop code, shop name 제외하는 것도 방법
- 어느 위치에 어떤 업종으로 어떤 객단가인 매장을 오픈하면 월매출이 어떻게 될까? 문제
 - X: shop type big, shop type small, longitude, latitude, avg_sale_price, 
 - y: montly_gain

In [ ]:
data.columns = ['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small', 
                'longitude', 'latitude', 'monthly_gain', 'average_sale_price', 'address1', 'address2']

In [ ]:
data.info()

In [ ]:
data

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
data[['monthly_gain', 'average_sale_price']].describe().transpose()

### shop_name, shop_code
- shop_code가 식별자 feature, 가게는 동일하지만 가게명이 바뀌는 경우가 존재함
    - shop_name(13,633), shop_code(13,352) -> 약 300 가게 정도는 이름이 바뀐 것으로 추정
    - 확인 결과 총 222가게가 이름을 최소 2번 바꾸었으며 평균 4번, 최대 66번 바꾸었음 
- 결론-> shop code, shop name 은 모델입력으로 적합하지 않음
- 업종 별 매출 예측으로만 봐야 적당할 것으로 보임

In [ ]:
# shop_name과 shop_code가 항상 같다면, 위 두 숫자가 같아야 하는데 약 300개의 data가 차이가 남 
# shop_code는 동일하지만(가게는 동일하지만) 업종, 가게명이 바뀌었을 수도 있음 -> shop_code를 식별자 feature로 사용

print(data['shop_name'].nunique())
print(data['shop_code'].nunique())
print(data['date'].unique())

In [ ]:
# shop code 를 group by 했을 때 그 안에 몇 종류의 shop name이 있는지 (1이어야 정상)

check_df = data.groupby('shop_code')['shop_name'].nunique().to_frame('shop_name_unique').reset_index()
check_df

In [ ]:
# shop code안에 여러 shop name 있는 경우

check_df2 = check_df[check_df['shop_name_unique'] > 1]
check_df2

In [ ]:
# 총 222가게가 이름을 최소 2번 바꾸었으며 평균 4번, 최대 66번 바꾸었음 

check_df2['shop_name_unique'].describe()

In [ ]:
# 롯데 월드 안에  놀이기구도 있지만, 편의점, 호텔 등 다는 업종 매출로 잡히는 것들도 있음!
# 결론-> shop code, shop name 은 모델입력으로 적합하지 않음
# 업종 별 매출 예측으로만 봐야 적당할 것으로 보임
data[data['shop_code'] == 148500219]

In [ ]:
# date column 분석
# shop_cde가 점포 하나하나를 식별하는 feature라면, 모든 각각의 점포가 24개의 date data를 가지고 있는지 확인  
check_df3 = data.groupby('shop_code')['date'].nunique().to_frame('date_unique').reset_index()
check_df3['date_unique'].describe()

In [ ]:
check_df3['date_unique'].value_counts().sort_values()

In [ ]:
data['date'].unique()

In [ ]:
# 1개의 date 데이터만 가진 점포 데이터가 5092개가 존재
# 2개의 date 데이터만 가진 점포 데이터가 5026개 존재 
# 3 -> 4674, 4->273 .,,, 
# 가정1) 오래 장사를 한 점포일수록 매출이 높을 것이다, date 개수가 많을 수록 monthly_gain.sum()이 높아질 것  
# 가정2) date가 1개인 점포는 매출데이터를 신뢰할 수 있을까? 2018년2월데이터가 아닌 2017년 3월데이터(시작날짜)라면 개점하자마자 폐점?
# 하지만 date 데이터를 1~12로 새로운 column으로 넣어준다 하더라도...
# 상점 하나하나가 아닌 월 매출 데이터 하나하나를 보는 것이기 때문에 넣기가 어려울 것 같음 
check_df3[check_df3['date_unique']<4]

### Preprocessing for more eda...

In [ ]:
# 매출데이터에 0값은 없음 -> 정확한 eda를 위해서 이를 제거하고 계속 진행하겠음 
preprocessed = data[data.monthly_gain!=0].copy()
#preprocessed

In [ ]:
print(preprocessed['shop_type_big'].unique())
print(len(preprocessed['shop_type_big'].unique()))
print(preprocessed['shop_type_big'].isnull().value_counts())

In [ ]:
null_index = preprocessed[preprocessed['shop_type_big'].isnull()==True].index
preprocessed = preprocessed.drop(null_index).reset_index().drop("index", axis=1)
preprocessed

In [ ]:
# 다행히도 shop_type_small에 대한 Nan값은 shop_type_big에서 제거함으로써 같이 제거된것으로 보입니다 
# preprocessed -> eda를 위해 전처리한 dataframe, data대신 해당 dataframe에서 쪼개서 eda를 진행합니다 
print(preprocessed['shop_type_small'].unique())
print(len(preprocessed['shop_type_small'].unique()))
print(preprocessed['shop_type_small'].isnull().value_counts())

### shop_type_big, shopt_type_small, monthly_gain
- 매출 데이터(monthly_gain)에 missing value 결측치 존재(3,605개) -> 제거하고 확인 총 149,789개 
- shop_type_big에 missing vlaue 결측치 존재(4,303개) -> 제거하고 확인 총 145,487개
- monthly_gain 
    - mean|     80,595,343.95 | 8천
    - min |               5.01| 
    - 25% |	     4,557,295.50 | 4백만
    - 50% |	    15,793,050.00 | 1천5백만
    - 75% |	    43,372,024.00 | 4천3백만
    - max | 65,341,444,752.55 | 653억4천1백만 (원래 1000억 넘게 나왔는데 결측치때문이였던 것 같음)
- average_sale_price
    - mean|    190,292.08
    - min |          1.00
    - 25% |     18,559.00 | 1천8백
    - 50% |     47,145.00 | 4천7백
    - 75% |    141,594.50 | 1억4천
    - max | 82,360,957.00 | 823억6천
- shop type big (13)
    - 음식(51004) 소매(47318) 의료(16618) 생활서비스(16074) 학문/교육(4495) 관광/여가/오락(3676) 도매/유통/무역(2607) 스포츠(1525) 숙박(1198) 제조(518) 부동산(335) 전자/정보통신(98) 기술서비스(21)
    - min 
    - '기술서비스', '전자/정보통신', '제조', '의료', '부동산', '숙박', '스포츠', '도매/유통/무역', '음식', '학문/교육', '관광/여가/오락', '소매', '생활서비스'
    - 부동산(10,060) 기준 이하(7곳)은 비슷함
    - mean 
    - '도매/유통/무역', '숙박', '제조', '전자/정보통신', '소매', '의료', '관광/여가/오락', '학문/교육', '음식', '생활서비스', '스포츠', '부동산', '기술서비스'
    - max 
    - '도매/유통/무역', '소매', '관광/여가/오락', '제조', '숙박', '의료', '전자/정보통신', '생활서비스', '학문/교육', '음식', '스포츠', '부동산', '기술서비스'
- 평균만 놓고 보아도 가장 낮은 업종은 천만원대, 가장 높은 업종은 억대이다 
- shop type small (367)


### monthly_gain

In [ ]:
df_check = preprocessed[['monthly_gain','shop_type_small','shop_type_big', 'average_sale_price']].copy()
# 데이터 값 실수. 소수점 두째자리까지 표시df
pd.set_option('display.float_format', '{:.2f}'.format) 

df_check

In [ ]:
pd.DataFrame(df_check['monthly_gain']).describe()

### average_sale_price

In [ ]:
pd.DataFrame(df_check['average_sale_price']).describe()

### shop_type_big

In [ ]:
group_big_gain = df_check.drop(['shop_type_small', 'average_sale_price'], axis=1).groupby('shop_type_big')
df_group_big_gain = group_big_gain.describe().droplevel(axis=1,level=0)

group_big_ave = df_check.drop(['shop_type_small', 'monthly_gain'], axis=1).groupby('shop_type_big')
df_group_big_ave = group_big_ave.describe().droplevel(axis=1,level=0)

# monthly_gain을 shop_type_big 분야별로 정렬하여 출력
print("내림차순 기준 분야별로 정렬하여 출력합니다 - monthly_gain")
print("min: ", list(df_group_big_gain.sort_values(by=['min'], ascending=False).index))
print("mean: ", list(df_group_big_gain.sort_values(by=['mean'], ascending=False).index))
print("max: ", list(df_group_big_gain.sort_values(by=['max'], ascending=False).index))

# average_sale_price을 shop_type_big 분야별로 정렬하여 출력
print("\n\n내림차순 기준 분야별로 정렬하여 출력합니다 - average_sale_price")
print("min: ", list(df_group_big_ave.sort_values(by=['min'], ascending=False).index))
print("mean: ", list(df_group_big_ave.sort_values(by=['mean'], ascending=False).index))
print("max: ", list(df_group_big_ave.sort_values(by=['max'], ascending=False).index))

In [ ]:
rcParams['figure.figsize'] = 15,8
sns.boxplot(x='shop_type_big', y='monthly_gain', data=df_check)

In [ ]:
rcParams['figure.figsize'] = 15,8
sns.boxplot(x='shop_type_big', y='average_sale_price', data=df_check)

In [ ]:
group_big_gain.boxplot(figsize=(12,16))

In [ ]:
group_big_ave.boxplot(figsize=(12,16))

In [ ]:
fig, axes = plt.subplots(1, 3)

df_group_big_gain.plot.bar(ax = axes[0], y='min')
df_group_big_gain.plot.bar(ax = axes[1], y='mean')
df_group_big_gain.plot.bar(ax = axes[2], y='max')

plt.rcParams['figure.figsize'] = [14, 6]
plt.show()

In [ ]:
group_big_gain.describe()

### shop_type_small

In [ ]:
#pd.set_option('display.max_rows', 14)
group_small = df_check.groupby('shop_type_small')
group_small.describe()

In [ ]:
# density plot only
ax = plt.subplots()
ax = sns.distplot(df_check['monthly_gain'], hist = False)
ax.set_title('Monthly gain Histogram with Density Plot')

In [ ]:
# density plot only
ax = plt.subplots()
ax = sns.distplot(df_check['average_sale_price'], hist = False)
ax.set_title('Average sale price Histogram with Density Plot')

### average sale price, monthly gain correlation 
- 따로 단가와 매출의 상관관계를 분석합니다

In [ ]:
# LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_check['shop_type_big'] = le.fit_transform(list(df_check['shop_type_big']))   # fit transform으로 한번에 처리 가능

le = LabelEncoder()

df_check['shop_type_small'] = le.fit_transform(list(df_check['shop_type_small'])) 
#le.classes_
df_check

In [ ]:
corr = df_check.corr()

sns.clustermap(corr, 
               annot = True,      # 실제 값 화면에 나타내기
               cmap = 'RdYlBu_r',  # Red, Yellow, Blue 색상으로 표시
               vmin = -1, vmax = 1, #컬러차트 -1 ~ 1 범위로 표시
              )

### longitude, latitude 
- longitude 경도(세로)
- latitude 위도(가로)
- Clustering 결과 대략 9개의 클러스터로 나뉘는 것을 알 수 있음
- 각 클러스터별 데이터 개수, monthly_gain의 평균값을 오름차순으로 정리한 것은 다음과 같음
- count:  [7, 8, 6, 0, 5, 4, 3, 2, 1]
- mean:  [5, 2, 3, 4, 1, 0, 6, 8, 7]

In [ ]:
pd.set_option('display.float_format', None)
df_geo = preprocessed[['longitude', 'latitude', 'monthly_gain']].copy()

drop_longitude = df_geo[df_geo['longitude']==0].index 
drop_latitude = df_geo[df_geo['latitude']==0].index 
print(len(drop_longitude))
print(len(drop_latitude))

In [ ]:
df_geo.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(x='longitude',y='latitude',data=df_geo)

In [ ]:
from sklearn.cluster import KMeans

#points = df_gps[['latitude', 'longitude']].values
kmeans = KMeans(n_clusters=9).fit(df_geo[['latitude', 'longitude']])
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
df_geo['cluster'] = kmeans.labels_
df_geo

In [ ]:
sns.scatterplot(x='longitude' , y='latitude', hue="cluster", data=df_geo, palette="Paired")
plt.title('k-mean')

In [ ]:
for i in range(9):
    labels=df_geo[df_geo["cluster"]==i]
    plt.scatter(labels['longitude'],labels['latitude'],label=i)

plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.show()

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format) 
group_geo = df_geo.groupby('cluster')
group_geo['monthly_gain'].describe()

In [ ]:
print("오름차순 기준 클러스터별로 정렬하여 출력합니다")
print("count: ", list(group_geo['monthly_gain'].describe().sort_values(by=['count']).index))
print("mean: ", list(group_geo['monthly_gain'].describe().sort_values(by=['mean']).index))
group_geo['monthly_gain'].describe().sort_values(by=['count'])

In [ ]:
# 클러스터별 매출 총 합산액
group_geo['monthly_gain'].sum().plot.bar()

### 성별, 요일, 시간 
- 확인 안하면 큰일날 뻔 했다... 
- 성별에 0만 가득찬 데이터가 1,187개 존재 
- 제거하면 총 144,300개 data

In [ ]:
pd.set_option('display.float_format', None)

df_gender = preprocessed[[ 
       'female_20', 'female_30', 'female_40', 'female_50', 'male_20',
       'male_30', 'male_40', 'male_50']].copy()
df_time = preprocessed[[
    'weekday0', 'weekday1', 'weekday2',
       'weekday3', 'weekday4', 'weekday5', 'weekday6']].copy()
df_week = preprocessed[['time_05_10', 'time_10_14', 'time_14_18',
            'time_18_20', 'time_20_22', 'time_22_24', 'time_00_05']].copy()

In [ ]:
df_gender.describe()

In [ ]:
df_time.describe()

In [ ]:
df_week.describe()

In [ ]:
df_gender.sum(axis=1).describe()

In [ ]:
# gender 열에 0.0으로만 가득한 데이터가 1,187개가 존재 
gender_index = df_gender[df_gender.sum(axis=1)==0].index

print(144300+1187)
df_gender=df_gender.drop(gender_index)
df_gender

In [ ]:
df_gender.sum(axis=1).describe()